# Exploratory Data Analysis and Deep Learning on Indian Housing Data
#### In this notebook I am going to be taking a look at data on rentla properties in 6 major cities in India in 2022. The data was taken from the following link: https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset

In [1]:
import numpy as np
import pandas as pd

# Getting the data

DataSet = pd.read_csv('../input/house-rent-prediction-dataset/House_Rent_Dataset.csv')

### Let's visualize the data by looking at the rent distribution in each city. I chose to plot the Log of the rent so that the large values dont dominate

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

DataSet['LogRent'] = np.log10(DataSet['Rent'])
DataSet['LogSize'] = np.log10(DataSet['Size'])
sns.violinplot(data = DataSet,x='City',y='LogRent')
plt.title('Violin Plot of Rent ')
plt.show()

#### From the violin plot, we can see how each city has a unique distribution. For example, we can see that the rental market in Bangalore is comprised mostly of lower rent homes, but has some properties going for vastly higher rents. In general, the cheapest city to live in is Kolkata and the most expensive city is Mumbai which is more or less in line with what we would expect

### Let's make another visualization to see how floor space relates to rent prices

In [3]:
sns.jointplot(data=DataSet,x='LogSize',y='LogRent',hue='City')
plt.show()

#### From this we can see that Delhi has a bimodal distribution, so we will take a closer look

In [4]:
g = sns.jointplot(data=DataSet[DataSet['City'] == 'Delhi'],x='LogSize',y='LogRent',kind='hist')
g.plot_joint(sns.kdeplot, color="r", zorder=1, levels=4)
g.fig.suptitle('Housing Data in Mumbai',size='xx-large',va='bottom')
plt.show()

#### Just as we thought, there seems to be two different trends going on here. Let's see if we can use unsupervised learning to separate the two sets of data

## Hierarchical Clustering
### We can use hierarchical clustering to create a dendrogram showing how our data points are related

In [5]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import matplotlib.pyplot as plt


link_mat = linkage(DataSet[DataSet['City'] == 'Delhi'][['LogRent','LogSize']], 'ward') # This creates the linkage, aka the hierarchy

plt.title('Hierarchical Clustering Dendrogram') # Plotting the dendrogram 
plt.axis('off') # Removing the axes because labels arent important and they overlap
dendrogram(link_mat,truncate_mode='lastp',p=200) # Alter p to show a truncated diagram
plt.show()

#### From the dendrogram, we can see that there is strong evidence for either 2 or 4 different natural clusters in the Delhi data

### Let's now plot our data again using two clusters to see if we have successfully separated the two trends that existed in our data

In [6]:
n_clusters = 4 # Number of classes to get labels for
h_label = fcluster(link_mat,t=n_clusters,criterion='maxclust') # Getting the class labels

for i in range(len(h_label)):
    if h_label[i] != 4:
        h_label[i] = 0
    else:
        h_label[i] = 1

g = sns.jointplot(data=DataSet[DataSet['City'] == 'Delhi'],x='LogSize',y='LogRent',hue=h_label)
g.fig.suptitle('Housing Data in Mumbai',size='xx-large',va='bottom')
plt.show()

#### We were indeed successful, and now instead of having one bimodal distribution we have separated our data into two unimodal distributions

# Machine Learning for Rent Prediction
### Now that we have looked at our data, let's see if we can use deep learning in order to predict rental prices given a description of the property. The first step toward doing this is feature engineering/data cleaning.

#### The first thing I would like to do is convert the floor data from being nominal data containing strings to two sets of ordinal data. The way that the 'Floor' data is stored as a string saying 'floor the apartment is on' out of 'number of floors in the building'. Often, the floor is indicated by the word 'Ground' or 'Upper'. I will have to deal with these cases as well. Finally, I will add a column in our dataframe called 'Height' which gives the ration between the floor the apartment is on and the number of floors the building has.

In [7]:
floor = []; numfloor = [];
for element in DataSet['Floor']:
    floor.append(element.split()[0])
    numfloor.append(element.split()[-1])

DataSet['NumberFloors'] = numfloor
DataSet['Level'] = floor
DataSet['NumberFloors'] = DataSet['NumberFloors'].replace(to_replace='Ground',value=0)
DataSet['Level'] = DataSet['Level'].replace(to_replace='Ground',value=0)
DataSet['Level'] = DataSet['Level'].replace(to_replace='Lower',value=0)
DataSet.loc[DataSet['Level'] == 'Upper', 'Level'] = DataSet['NumberFloors']
DataSet['Level'] = pd.to_numeric(DataSet['Level']) + 1
DataSet['NumberFloors'] = pd.to_numeric(DataSet['NumberFloors']) + 1
DataSet['Height'] = DataSet['Level']/DataSet['NumberFloors']

### Time to build our model. We will create a feedforward deep neural network with batch normalization to assist in training. In order to deal with all of the categorical data the we have, we will one-hot encode our data. Finally, we will separate our data into train, test, and validation sets and then scale our data.

In [8]:
import tensorflow as tf
print("num GPUs available:", len(tf.config.experimental.list_physical_devices("GPU")))
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

# Initializing scalers

scaler = RobustScaler()
scaler2 = StandardScaler()

# Performing one-hot encoding

DataSet2 = pd.get_dummies(DataSet, columns=['Area Type','Area Locality','City','Furnishing Status','Tenant Preferred'])

# Removing outliers

q_high = DataSet2['Rent'].quantile(0.99)
q_low = DataSet2['Rent'].quantile(0.01)
DataSet2 = DataSet2[(DataSet2['Rent'] < q_high) & (DataSet2['Rent'] > q_low)]

# Forming train and test sets

X = DataSet2.drop(columns = ['Rent','Floor','LogRent','LogSize','Point of Contact','Posted On'])
y = DataSet2['Rent']

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
y_train_df = pd.DataFrame(data=y_train)

# Scaling the data

x_train_scaled = scaler.fit_transform(x_train)
y_train_scaled = scaler2.fit_transform(y_train_df)

x_train, x_val, y_train, y_val = train_test_split(x_train_scaled,y_train_scaled, test_size = 0.2)


In [9]:
# Building the model

MSE_model = Sequential()
MSE_model.add(Dense(256, activation = 'relu', input_dim = len(X.columns)))
MSE_model.add(Dense(128))
MSE_model.add(BatchNormalization())
MSE_model.add(Activation('relu'))
MSE_model.add(Dense(128))
MSE_model.add(BatchNormalization())
MSE_model.add(Activation('relu'))
MSE_model.add(Dense(128))
MSE_model.add(BatchNormalization())
MSE_model.add(Activation('relu'))
MSE_model.add(Dense(1, activation = 'linear'))
MSE_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_percentage_error'])
MSE_model.summary()

# Getting the history to plot the training

history = MSE_model.fit(x_train_scaled, y_train_scaled, epochs=100,validation_data=(x_val,y_val), verbose = False)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Training','Validation'])
plt.show()

#### As we can see, the training stabilizes at around 100 epochs, and we don't see a divergence between the training and validation data

In [10]:
pred = scaler2.inverse_transform(MSE_model.predict(scaler.transform(x_test)))
real = y_test
pred_df = pd.DataFrame(data=pred)
real_df = pd.DataFrame(data=real)
real_df.reset_index(inplace=True)
real_df.drop(columns = ['index'],inplace = True)
print('The Mean Absolute Percentage Error is:',sum(abs(real_df['Rent'] - pred_df[0]))/real_df['Rent'].sum())

### We know the Mean Absolute Percentage Error of our model, but it is important to visualize or results to make sure we are aware of the drawbacks of our model

In [11]:
ax = sns.histplot((real_df['Rent'] - pred_df[0])/real_df['Rent']*100,kde=True)
ax.lines[0].set_color('crimson')
plt.title('Histogram of Rent Predictions')
plt.xlabel('Mean Percentage Error (%)')
plt.show()

#### As we can see, most of our predictions are on the mark, but our distribution is left-skewed. This means that sometimes we are predicting that rents will be much higher than they actually are. This is probably due to the fact that in each city, a small amount of apartments will be far more expensive than the rest. We trained our network using MSE which will heavily punish these errors. Therefore our network will approach a weighted geometric mean between these high rents and the more common low rent values, giving us a left-skewed distribution

### In order to mitigate the issue with over-estimating rents, I will do more feature engineering. Perhaps we can capture the cases when rent is much higher than expected by clustering our data and adding the cluster labels as a feature

#### Last time we used hierarchical clustering, but this time we will do k-means.

In [12]:
import sklearn.cluster as cluster

kmeans = cluster.KMeans(n_clusters = 18, init = 'k-means++')
kmeans = kmeans.fit(DataSet[['Rent','Size','BHK','Level','NumberFloors']])

DataSet['Cluster'] = kmeans.labels_

In [13]:
import tensorflow as tf
print("num GPUs available:", len(tf.config.experimental.list_physical_devices("GPU")))
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

# Initializing scalers

scaler = RobustScaler()
scaler2 = StandardScaler()

# Performing one-hot encoding

DataSet2 = pd.get_dummies(DataSet, columns=['Area Type','Area Locality','City','Furnishing Status','Tenant Preferred'])

# Removing outliers

q_high = DataSet2['Rent'].quantile(0.99)
q_low = DataSet2['Rent'].quantile(0.01)
DataSet2 = DataSet2[(DataSet2['Rent'] < q_high) & (DataSet2['Rent'] > q_low)]

# Forming train and test sets

X = DataSet2.drop(columns = ['Rent','Floor','LogRent','LogSize','Point of Contact','Posted On'])
y = DataSet2['Rent']

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
y_train_df = pd.DataFrame(data=y_train)

# Scaling the data

x_train_scaled = scaler.fit_transform(x_train)
y_train_scaled = scaler2.fit_transform(y_train_df)

x_train, x_val, y_train, y_val = train_test_split(x_train_scaled,y_train_scaled, test_size = 0.2)

# Building the model

MSE_model2 = Sequential()
MSE_model2.add(Dense(256, activation = 'relu', input_dim = len(X.columns)))
MSE_model2.add(Dense(128))
MSE_model2.add(BatchNormalization())
MSE_model2.add(Activation('relu'))
MSE_model2.add(Dense(128))
MSE_model2.add(BatchNormalization())
MSE_model2.add(Activation('relu'))
MSE_model2.add(Dense(128))
MSE_model2.add(BatchNormalization())
MSE_model2.add(Activation('relu'))
MSE_model2.add(Dense(1, activation = 'linear'))
MSE_model2.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_percentage_error'])
MSE_model2.summary()

# Getting the history to plot the training

history = MSE_model2.fit(x_train_scaled, y_train_scaled, epochs=100,validation_data=(x_val,y_val), verbose = False)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Training','Validation'])
plt.show()

In [14]:
pred2 = scaler2.inverse_transform(MSE_model2.predict(scaler.transform(x_test)))
real2 = y_test
pred_df2 = pd.DataFrame(data=pred2)
real_df2 = pd.DataFrame(data=real2)
real_df2.reset_index(inplace=True)
real_df2.drop(columns = ['index'],inplace = True)
print('The Mean Absolute Percentage Error When Trained with Cluster Labels is:',sum(abs(real_df2['Rent'] - pred_df2[0]))/real_df2['Rent'].sum())

df1 = pd.DataFrame(data=(real_df['Rent'] - pred_df[0])/real_df['Rent']*100)
df2 =  pd.DataFrame((real_df2['Rent'] - pred_df2[0])/real_df2['Rent']*100)
df1['label'] = np.zeros(len(real_df))
df2['label'] = np.ones(len(real_df2))
frames = [df1, df2]
df = pd.concat(frames,ignore_index=True)
df['label'].replace(0,'Original Model', inplace = True)
df['label'].replace(1,'Trained on Clustered Data', inplace = True)

sns.displot(data=df,x=0,hue='label',kind='kde')
plt.title('Histogram of Rent Predictions')
plt.xlabel('Mean Percentage Error (%)')
plt.show()

### As we hoped, the clustering did in fact help. We have a much stronger central peak and fewer points in the over-estimate region

In [ ]:
from sklearn import model_selection
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn import metrics

scaler3 = StandardScaler()

x_train_svr, x_test_svr, y_train_svr, y_test_svr = train_test_split(X,y, test_size = 0.2)
y_train_svr_df = pd.DataFrame(data=y_train_svr)
y_test_svr_df = pd.DataFrame(data=y_test_svr)
y_train_svr_scaled = scaler3.fit_transform(y_train_svr_df)
y_test_svr_scaled = scaler3.transform(y_test_svr_df)

my_pipe = Pipeline([('scaler', StandardScaler()), ('regressor',SVR())])

params = [{'scaler'                   : [MinMaxScaler(), StandardScaler(), RobustScaler()],
           'regressor__kernel'       : ['poly', 'rbf'],
           'regressor__degree'       : [1,2,3,4],
           'regressor__C'            : [0.5, 0.8, 1.0, 2.0]}]

grid = GridSearchCV(my_pipe, params, cv=5, scoring = 'neg_root_mean_squared_error')
grid.fit(x_train_svr, y_train_svr_scaled.ravel())

# Reporting best parameters

print(grid.best_params_)
print(grid.score(x_test, y_test.ravel()))

In [50]:
svr = SVR(degree=2,kernel='poly',C=1)
svr.fit(x_train, y_train)
svr.score(x_val,y_val)

In [51]:
pred3 = scaler2.inverse_transform((svr.predict(scaler.transform(x_test))).reshape(-1, 1))
real3 = y_test
pred_df3 = pd.DataFrame(data=pred3)
real_df3 = pd.DataFrame(data=real3)
real_df3.reset_index(inplace=True)
real_df3.drop(columns = ['index'],inplace = True)
print('The Mean Absolute Percentage Error When Trained with Cluster Labels and SVR is:',sum(abs(real_df3['Rent'] - pred_df3[0]))/real_df3['Rent'].sum())


df1 = pd.DataFrame(data=(real_df['Rent'] - pred_df[0])/real_df['Rent']*100)
df2 =  pd.DataFrame((real_df2['Rent'] - pred_df2[0])/real_df2['Rent']*100)
df3 =  pd.DataFrame((real_df3['Rent'] - pred_df3[0])/real_df3['Rent']*100)
df1['label'] = np.zeros(len(real_df))
df2['label'] = np.ones(len(real_df2))
df3['label'] = 2*np.ones(len(real_df3))
frames = [df1, df2, df3]
df = pd.concat(frames,ignore_index=True)
df['label'].replace(0,'Original Model', inplace = True)
df['label'].replace(1,'Trained on Clustered Data', inplace = True)
df['label'].replace(2,'SVR Trained on Clustered Data', inplace = True)

sns.displot(data=df,x=0,hue='label',kind='kde')
plt.title('Histogram of Rent Predictions')
plt.xlabel('Mean Percentage Error (%)')
plt.show()

### Can we do more feature engineering?

#### When we did our one-hot encoding, the area localities feature had over 2000 unique entries. This is a lot of features and we end up with a quite sparse matrix. Instead of naively one-hot encoding the area localities, we can instead try to group the localities into k clusters and one-hot encode based on the k labels


### To cluster based on Area Locality, I first make a pivot table to aggregate the data for each locality

In [ ]:
DataSet.drop(columns = ['Cluster'],inplace=True)
data_pivot = pd.pivot_table(data=DataSet[['BHK','Rent','Size','Bathroom','Level','NumberFloors','Area Locality']],index='Area Locality')
data_pivot

### We then perform the clustering. I chose to use 60 clusters, perhaps corresponding to 10 for each city

In [ ]:
scaler3 = RobustScaler()
data_pivot_scaled = scaler3.fit_transform(data_pivot)
kmeans2 = cluster.KMeans(n_clusters = 60, init = 'k-means++')
kmeans2 = kmeans2.fit(data_pivot_scaled)

data_pivot['Cluster'] = kmeans2.labels_

sns.countplot(data=data_pivot,x='Cluster',order = data_pivot['Cluster'].value_counts().index)
plt.xticks([])
plt.plot()

#### I plotted the counts for each cluster to ensure not all localities were clustered to only a few centroids

In [ ]:
DataSet_clust = DataSet.drop(columns = ['Posted On','LogSize','LogRent','Height','Floor'])
for i in range(len(data_pivot.index)):
    DataSet_clust['Area Locality'] = DataSet_clust['Area Locality'].replace(to_replace=data_pivot.index[i],value=data_pivot['Cluster'][i])
    

scaler = RobustScaler()
scaler2 = StandardScaler()

# Performing one-hot encoding

DataSet3 = pd.get_dummies(DataSet_clust, columns=['Area Type','Area Locality','City','Furnishing Status','Tenant Preferred'])

# Removing outliers

q_high = DataSet3['Rent'].quantile(0.99)
q_low = DataSet3['Rent'].quantile(0.01)
DataSet3 = DataSet3[(DataSet3['Rent'] < q_high) & (DataSet3['Rent'] > q_low)]

# Forming train and test sets

X = DataSet3.drop(columns = ['Rent','Point of Contact'])
y = DataSet2['Rent']

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
y_train_df = pd.DataFrame(data=y_train)

# Scaling the data

x_train_scaled = scaler.fit_transform(x_train)
y_train_scaled = scaler2.fit_transform(y_train_df)

x_train, x_val, y_train, y_val = train_test_split(x_train_scaled,y_train_scaled, test_size = 0.2)

# Building the model

MSE_model3 = Sequential()
MSE_model3.add(Dense(256, activation = 'relu', input_dim = len(X.columns)))
MSE_model3.add(Dense(128))
MSE_model3.add(BatchNormalization())
MSE_model3.add(Activation('relu'))
MSE_model3.add(Dense(128))
MSE_model3.add(BatchNormalization())
MSE_model3.add(Activation('relu'))
MSE_model3.add(Dense(128))
MSE_model3.add(BatchNormalization())
MSE_model3.add(Activation('relu'))
MSE_model3.add(Dense(1, activation = 'linear'))
MSE_model3.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_percentage_error'])
MSE_model3.summary()

# Getting the history to plot the training

history = MSE_model3.fit(x_train_scaled, y_train_scaled, epochs=100,validation_data=(x_val,y_val), verbose = False)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(['Training','Validation'])
plt.show()

#### As with previous models, our training stabilizes around 100 epochs with no major diversion between train and validation.

In [ ]:
pred3 = scaler2.inverse_transform(MSE_model3.predict(scaler.transform(x_test)))
real3 = y_test
pred_df3 = pd.DataFrame(data=pred3)
real_df3 = pd.DataFrame(data=real3)
real_df3.reset_index(inplace=True)
real_df3.drop(columns = ['index'],inplace = True)
print('The Mean Absolute Percentage Error is:',sum(abs(real_df3['Rent'] - pred_df3[0]))/real_df3['Rent'].sum())

### Finally, we will compare the original model with the naive one-hot encoding, the model with one-hot encoding as well as an added feature of labels from k-means clustering, and the new model we just made

In [ ]:
df1 = pd.DataFrame(data=(real_df['Rent'] - pred_df[0])/real_df['Rent']*100)
df2 =  pd.DataFrame((real_df2['Rent'] - pred_df2[0])/real_df2['Rent']*100)
df3 =  pd.DataFrame((real_df3['Rent'] - pred_df3[0])/real_df3['Rent']*100)
df1['label'] = np.zeros(len(real_df))
df2['label'] = np.ones(len(real_df2))
df3['label'] = np.ones(len(real_df2))*2
frames = [df1, df2, df3]
df = pd.concat(frames,ignore_index=True)
df['label'].replace(0,'Original Model', inplace = True)
df['label'].replace(1,'Trained on Clustered Data', inplace = True)
df['label'].replace(2,'Trained with Clustered Localities', inplace = True)

sns.displot(data=df,x=0,hue='label',kind='kde')
plt.title('Histogram of Rent Predictions')
plt.xlabel('Mean Percentage Error (%)')
plt.show()

### As you can see, our new model underperforms compared to the other models. This is not surprising given the fact that we have far less features to learn on as compared to the other models. However, if computation time is a larger factor, perhaps the reduction in accuracy is worth it here